<a href="https://colab.research.google.com/github/gajanan-nawle/Flipkart-CSAT-GenAI-Azure/blob/main/Capstone_Project_Machine_Learning_%26_GenAI_with_Microsoft_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
# Configuration
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

# **2. Load Dataset**

In [ ]:
df = pd.read_csv('Customer_support_data.csv')

# **3. Initial Data Inspection**

In [ ]:
#column and row count
print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")

Shape: 85907 rows, 20 columns


In [ ]:
#Viewing first 5 rows to understand data format
print(df.head())


                              Unique id channel_name         category  \
0  7e9ae164-6a8b-4521-a2d4-58f7c9fff13f      Outcall  Product Queries   
1  b07ec1b0-f376-43b6-86df-ec03da3b2e16      Outcall  Product Queries   
2  200814dd-27c7-4149-ba2b-bd3af3092880      Inbound    Order Related   
3  eb0d3e53-c1ca-42d3-8486-e42c8d622135      Inbound          Returns   
4  ba903143-1e54-406c-b969-46c52f92e5df      Inbound     Cancellation   

                   Sub-category Customer Remarks  \
0                Life Insurance              NaN   
1  Product Specific Information              NaN   
2             Installation/demo              NaN   
3        Reverse Pickup Enquiry              NaN   
4                    Not Needed              NaN   

                               Order_id order_date_time Issue_reported at  \
0  c27c9bb4-fa36-4140-9f1f-21009254ffdb             NaN  01/08/2023 11:13   
1  d406b0c7-ce17-4654-b9de-f08d421254bd             NaN  01/08/2023 12:52   
2  c273368d-b961-

In [ ]:
#Checking Data Types and Non-Null counts
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unique id                85907 non-null  object 
 1   channel_name             85907 non-null  object 
 2   category                 85907 non-null  object 
 3   Sub-category             85907 non-null  object 
 4   Customer Remarks         28742 non-null  object 
 5   Order_id                 67675 non-null  object 
 6   order_date_time          17214 non-null  object 
 7   Issue_reported at        85907 non-null  object 
 8   issue_responded          85907 non-null  object 
 9   Survey_response_Date     85907 non-null  object 
 10  Customer_City            17079 non-null  object 
 11  Product_category         17196 non-null  object 
 12  Item_price               17206 non-null  float64
 13  connected_handling_time  242 non-null    float64
 14  Agent_name            

In [ ]:
#Checking for Duplicates
duplicates = df.duplicated(subset=['Unique id']).sum()
print(f"Duplicate Records: {duplicates}")

Duplicate Records: 0


In [ ]:
#Statistical Summary of Numerical Columns
print("Numerical Summary")
display(df.describe())

In [ ]:
#Checking for Class Imbalance (Target Variable)
print("CSAT Score Distribution (Target)")
print(df['CSAT Score'].value_counts(normalize=True) * 100)